# Diversity and Ability
This jupyter notebook demonstrates how the agent-based model (ABM) of group decision-making by walking you through the stages of the model. The idea is that the group faces a binary choice problem and each individual has access to a number of imperfect sources to guide their individual decision-making. The main components of the agent-based model are: 

1. **Sources.** Each ABM consists of a set of sources. Each source has a certain reliability (typically, $\geq 0.5$), which represents the probability that it produces evidence that supports the correct alternative.  
2. **Agents.** Each agent in the ABM is connected to a number of sources, which determine the agent's decisions. The basic idea is that agents follow the majority of the evidences available to them. 
3. **Teams.** A team is a set of agents. The team decision-making follows the majority rule, so that the team picks the alternative that most team members think is best. 

In particular, an instantiation of the agent-based model, called `Community`, consists of the following building blocks:
- A set of sources $S$,
- For each source $s \in S$:
    - Its reliability $p_s$
- A set of agents $N$,
- For each agent $i \in N$:
    - A set of sources $S_i \subseteq S$ that she has access to. 

In future work, I plan to also include more complicated versions of group decision-making (other than majority rule) by including influence networks. 

## 0. Initialization
We start with loading some packages and scripts:

In [1]:
%run utils/notebook.py

## 1. A simple example
### 1.0 Initial stage
Let us start with considering a simple example of a community with only 20 agents. More explicitly, we set the parameters as follows:
- A set of 21 sources $S=\{s_1, \ldots, s_{21}\}$,
- A set of 21 agents $N=\{0, \ldots, 20\}$,
- For each agent $i \in N$:
    - Her sources $S_i \subseteq S$ (where $|S_i|=5$). 

The simple community is depicted in the picture below. Let me explain the picture:
- Nodes:
    - The initial color of the nodes represents whether they are <font color="teal">sources</font> or <font color="darkorange">agents</font>.
    - <font color="teal">Sources</font>:
        - The size of the sources is proportional to their reliability, so that bigger sources are more reliable.
        - The sources are ordered from left to right from most to least accessed. Equally accessed sources are ordered from left to right from most to least reliable. 
    - <font color="darkorange">Agents</font>:
        - The size and ordering of the agents conveys no information. 
- Edges:
    - The *edges* represent the agents' **sources**: these are the set of target sources of all outgoing edges. Accordingly, every agent has out-degree 5.
    - Hence, sources with higher in-degrees have more influence on the team decision-making. 
    - The color of an edge represents the color of the associated source. 
    
All the pictures are **interactive**: if you click on a source or agent, the connected nodes are highlighted, and you can move nodes around!

In [2]:
community = Community(
    number_of_agents=7,
    number_of_sources=21,
    sources_reliability_distribution=(0.5, 0.7),
    source_degree=5,
    influence_degree=4,
)
community.vote()

-1

In [3]:
visualize(community, coloring="initial").show("www/community_initial.html")

www/community_initial.html


### 1.1 Source valences
In the first stage, the valence of each source $s\in S$ is probabilistically determined by its reliability $p_s$. The color of the *sources* represents whether their piece of **evidence** has the <font color="forestgreen">**correct**</font> or <font color="firebrick">**incorrect**</font> valence, i.e., supports <font color="forestgreen">**correct**</font> or <font color="firebrick">**incorrect**</font> alternative.

In the second stage, each agent $i$ forms their opinion based on the evidences from their sources $S_i$. The color of the *agents* represents whether their **opinion** supports the <font color="forestgreen">**correct**</font> or <font color="firebrick">**incorrect**</font> alternative.

In the third stage, the team forms their collective decision by following the majority rule. 

Let's see how this goes in the artificial community. Did any source have the wrong valence? Can you figure out why? Can you predict some of the agent's opinions?

In [4]:
visualize(com=community, coloring="sources", ).show("www/community_sources.html")

www/community_sources.html


### 1.2 Agent opinions
Let us consider the opinions of the agents in this artificial community. As mentioned before, the color of the *agents* represents whether their **opinion** favours the <font color="forestgreen">**correct**</font> or <font color="firebrick">**incorrect**</font> alternative.

Let's see how this goes in the simple community. Did anyone form the wrong opinion? Can you figure out why?

In [5]:
visualize(com=community, coloring="agents").show("www/community_agents.html")

www/community_agents.html


### 1.3 The group decision
The outcome is probably no surprise to you at this point, but let's determine the team's collective decision:

In [6]:
correct_opinions = sum(
    [1 for agent in community.agents if community.influence_network.nodes[agent][cfg.agent_opinion] == cfg.vote_for_positive]
)
incorrect_opinions = community.number_of_agents - correct_opinions
print(f"Correct: {correct_opinions} | Incorrect: {incorrect_opinions}")

Correct: 1 | Incorrect: 6


## 2 Different teams
In this repository we consider three types of teams:
1. Teams consisting of the best-performing agents;
2. Teams consisting of a (cognitively) diverse set of agents;
3. Teams consisting of a randomly selected agents. 

As mentioned earlier, each agent is associated with a set of sources. We assume that all agents access the same number of sources, typically 5.

Instead of investigating a particular community, these teams are constructed for a given set of sources and their reliabilities.  More precisely, the task for constructing the team of best-performing agents is as follows:

```commandline 
Input 1: A set of sources and their reliabilities.
Input 2: A group size.
Input 3: A number of sources that each agent accesses.
---
> Output: The team of the required size consisting of the best-performing agents out of all possible agents.
```

Once a team has been constructed, the team decision-making works as illustrated in Section 1. 

### 2.1 The best team
Below you can see what the best team may look like. Notice that the members of the best team only have access to a small number of sources, namely the sources with the highest reliability. 

In [7]:
team = best_team(community, group_size=7)
visualize(team).show("www/best_team.html")

www/best_team.html


### 2.2 The diverse team
Below you can see what the most diverse team might look like. Notice that the most diverse team has access to all (or virtually all) sources &mdash; regardless of their reliability.

In [8]:
team = diverse_team(community, group_size=7)
visualize(team).show("www/diverse_team.html")

www/diverse_team.html


### 2.3 A random team
Below you can see what a random team might look like. Notice that a random team has access to many (but typically not all) sources &mdash; regardless of their reliability.

In [9]:
team = random_team(community, group_size=7)
visualize(team).show("www/random_team.html")

www/random_team.html


## 3. Thanks
I hope this brief illustration helped to understand the inner workings of the agent-based model. Thanks for showing interest!